In [2]:
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()

try:
    import sklearn
    print(sklearn.__version__)
    if (sklearn.__version__ < '1.3.0'): #update for latest scikit-learn version
        !pip install -U --user scikit-learn
except Exception as e:
    !pip install scikit-learn

1.3.0


In [3]:
regular_season = pd.read_csv("Consolidated Regular Season Data.csv")
regular_season.head()

,Rk,Team,Season (end),Age,W,L,PW,PL,MOV,SOS,...,FT/FGA,Playoffs,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Successful,Arena,Attend.,Attend./G
0,1,Miami Heat*,2011,29.0,58,24,61,21,7.46,-0.71,...,0.279,1.0,0.475,12.6,75.5,0.223,1,AmericanAirlines Arena,"810,930","19,779"
1,2,Chicago Bulls*,2011,26.4,62,20,61,21,7.32,-0.79,...,0.227,1.0,0.463,13.6,76.2,0.222,1,United Center,"893,462","21,792"
2,3,Los Angeles Lakers*,2011,30.2,57,25,58,24,6.11,-0.10,...,0.228,1.0,0.477,12.8,72.3,0.181,0,STAPLES Center,"778,877","18,997"
3,4,San Antonio Spurs*,2011,28.8,61,21,56,26,5.71,0.16,...,0.229,1.0,0.491,12.6,74.6,0.190,0,AT&T Center,"750,879","18,314"
4,5,Boston Celtics*,2011,29.5,56,26,57,25,5.37,-0.54,...,0.235,1.0,0.469,14.8,74.7,0.235,0,TD Garden,"763,584","18,624"


In [9]:
playoffs = pd.read_csv("Consolidated Playoffs - Sheet1.csv")
playoffs.head()

,Playoff Round,Winner,Loser,Year
0,Finals,Denver Nuggets,Miami Heat,2023
1,Eastern Conference Finals,Miami Heat,Boston Celtics,2023
2,Western Conference Finals,Denver Nuggets,Los Angeles Lakers,2023
3,Eastern Conference Semifinals,Boston Celtics,Philadelphia 76ers,2023
4,Eastern Conference Semifinals,Miami Heat,New York Knicks,2023


In [20]:
total_nba = regular_season

In [44]:
# removes the * from playoff teams
regular_season['Team'] = regular_season['Team'].str.replace('*', '', regex=False) 

playoffs['Year'] = playoffs['Year'].astype(int) 
regular_season['Season (end)'] = regular_season['Season (end)'].astype(int) 



In [67]:
regular_season['Team'] = regular_season['Team'].str.replace('*', '', regex=False)

playoffs['Year'] = playoffs['Year'].astype(int)
regular_season['Season (end)'] = regular_season['Season (end)'].astype(int)

loser_merged = pd.merge(regular_season, playoffs, left_on=['Season (end)', 'Team'], right_on=['Year', 'Loser'], how='left')
loser_merged['Lost To'] = loser_merged['Winner']  # Who they lost to
loser_merged['Round made it to'] = loser_merged['Playoff Round']  # Round they reached

winner_merged = pd.merge(regular_season, playoffs, left_on=['Season (end)', 'Team'], right_on=['Year', 'Winner'], how='left')
winner_merged['Lost To'] = None  # Winners didn't lose in the round they won

total_nba = pd.concat([loser_merged, winner_merged]).drop_duplicates(subset=['Team', 'Season (end)'], keep='first')


In [68]:
# Correct the "Round made it to" for the champions by finding the highest playoff round the team won
def find_champions(row):
    if row['Playoff Round'] == 'Finals':
        return 'NBA Champion'
    return row['Round made it to']

total_nba['Round made it to'] = total_nba.apply(find_champions, axis=1)

# Fill NaN for teams that did not make it to the playoffs or no playoff data was merged
total_nba['Round made it to'].fillna('Did not make playoffs', inplace=True)
total_nba['Lost To'].fillna('None', inplace=True)

# Clean up columns
total_nba.drop(columns=['Year', 'Winner', 'Loser', 'Playoff Round'], inplace=True)

# Save the result or perform further analysis
total_nba

,Rk,Team,Season (end),Age,W,L,PW,PL,MOV,SOS,...,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Successful,Arena,Attend.,Attend./G,Lost To,Round made it to
0,1,Miami Heat,2011,29.0,58,24,61,21,7.46,-0.71,...,0.475,12.6,75.5,0.223,1,AmericanAirlines Arena,"810,930","19,779",Dallas Mavericks,NBA Champion
1,2,Chicago Bulls,2011,26.4,62,20,61,21,7.32,-0.79,...,0.463,13.6,76.2,0.222,1,United Center,"893,462","21,792",Miami Heat,Eastern Conference Finals
2,3,Los Angeles Lakers,2011,30.2,57,25,58,24,6.11,-0.10,...,0.477,12.8,72.3,0.181,0,STAPLES Center,"778,877","18,997",Dallas Mavericks,Western Conference Semifinals
3,4,San Antonio Spurs,2011,28.8,61,21,56,26,5.71,0.16,...,0.491,12.6,74.6,0.190,0,AT&T Center,"750,879","18,314",Memphis Grizzlies,Western Conference First Round
4,5,Boston Celtics,2011,29.5,56,26,57,25,5.37,-0.54,...,0.469,14.8,74.7,0.235,0,TD Garden,"763,584","18,624",Miami Heat,Eastern Conference Semifinals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,26,Portland Trail Blazers,2023,25.1,33,49,31,51,-4.01,0.05,...,0.563,12.1,74.9,0.217,0,Moda Center,"767,374","18,716",None,Did not make playoffs
386,27,Charlotte Hornets,2023,25.3,27,55,26,56,-6.24,0.35,...,0.544,12.5,75.5,0.211,0,Spectrum Center,"702,052","17,123",None,Did not make playoffs
387,28,Houston Rockets,2023,22.1,22,60,23,59,-7.85,0.24,...,0.564,11.8,75.8,0.218,0,Toyota Center,"668,865","16,314",None,Did not make playoffs
388,29,Detroit Pistons,2023,24.1,17,65,22,60,-8.22,0.49,...,0.557,11.9,74.0,0.231,0,Little Caesars Arena,"759,715","18,596",None,Did not make playoffs
